In [55]:
import pandas as pd
from click import progressbar
from modin.core.execution.modin_aqp import progress_bars
from sklearn.utils import resample
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.impute import SimpleImputer
import joblib
from xgboost import XGBClassifier
import optuna
from optuna.samplers import TPESampler


In [66]:
#load dataset
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [32]:
#rebalance dataset
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# balance mayority class with minority
sub_class = resample(class_0, replace=False, n_samples=(len(class_1) * 2), random_state=42)

# combine
df = pd.concat([sub_class, class_1])

In [67]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

In [68]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [69]:
#divide categorical and numerical to impute the values
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="constant", fill_value= 0)  #fill nan's as 0
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [70]:
#change object to category to use in the threelike model
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y ,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train ,random_state=42)


In [84]:

# Identificar columnas categóricas
cat_features = X_train.select_dtypes(include=['category', 'object']).columns.tolist()
# Calcula el ratio de desbalance
neg_samples = (y_train == 0).sum()
pos_samples = (y_train == 1).sum()
scale_pos_weight = neg_samples / pos_samples

catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'PRAUC',
    'custom_metric': ['AUC', 'PRAUC', 'Precision', 'Recall'],


    'scale_pos_weight': scale_pos_weight * 0.9,

    # Boosting
    'iterations': 4000,
    'learning_rate': 0.025,
    'depth': 6,

    # Regularización (compensa falta de rsm)
    'l2_leaf_reg': 8.0,
    'random_strength': 2,

    # Bootstrap
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 0.5,

    # Growing
    'grow_policy': 'SymmetricTree',
    'min_data_in_leaf': 30,

    # Categóricas
    'cat_features': cat_features,
    'one_hot_max_size': 10,

    # GPU
    'task_type': 'GPU',
    'devices': '0',

    # Control
    'random_seed': 42,
    'early_stopping_rounds': 250,
    'verbose': 100,
    'use_best_model': True,}

In [85]:

model = CatBoostClassifier(**catboost_params)
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=200
)

Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.6919976	test: 0.6948770	best: 0.6948770 (0)	total: 38.5ms	remaining: 2m 34s
200:	learn: 0.7513781	test: 0.7484969	best: 0.7484969 (200)	total: 6.17s	remaining: 1m 56s
400:	learn: 0.7689836	test: 0.7563667	best: 0.7563667 (400)	total: 12s	remaining: 1m 47s
600:	learn: 0.7828052	test: 0.7598352	best: 0.7598352 (600)	total: 17.8s	remaining: 1m 40s
800:	learn: 0.7951156	test: 0.7618715	best: 0.7618725 (797)	total: 23.5s	remaining: 1m 34s
1000:	learn: 0.8063809	test: 0.7632623	best: 0.7632623 (1000)	total: 29.4s	remaining: 1m 27s
1200:	learn: 0.8168054	test: 0.7638452	best: 0.7638616 (1194)	total: 35.1s	remaining: 1m 21s
1400:	learn: 0.8264073	test: 0.7641143	best: 0.7641715 (1331)	total: 40.9s	remaining: 1m 15s
1600:	learn: 0.8355490	test: 0.7642634	best: 0.7644083 (1578)	total: 46.7s	remaining: 1m 9s
1800:	learn: 0.8444464	test: 0.7645421	best: 0.7645922 (1675)	total: 52.5s	remaining: 1m 4s
2000:	learn: 0.8528519	test: 0.7644971	best: 0.7646550 (1965)	total: 58.3s	remaining: 5

In [83]:

# Probabilidades (idéntico conceptualmente)
y_test_proba = model.predict_proba(X_test)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_test, y_test_proba)

#optimal threshold based on expected loss
cost_FN = 10000  # default
cost_FP = 1000  # lost good client

thresholds = np.linspace(0, 1, 101)
losses = []

for t in thresholds:
    y_pred = (y_test_proba >= t).astype(int)

    FP = np.sum((y_test == 0) & (y_pred == 1))
    FN = np.sum((y_test == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_test_pred = (y_test_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_test, y_test_pred)

#Confusion matrix manual
TP = np.sum((y_test == 1) & (y_test_pred == 1))
FP = np.sum((y_test == 0) & (y_test_pred == 1))
TN = np.sum((y_test == 0) & (y_test_pred == 0))
FN = np.sum((y_test == 1) & (y_test_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f'Used Threshold: {optimal_threshold}')
print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_test, y_test_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

Used Threshold: 0.09
FPR: 0.2747
Precision (PPV): 0.1950
AUC validación: 0.78470
              precision    recall  f1-score   support

           0       0.96      0.73      0.83     46061
           1       0.20      0.70      0.31      4367

    accuracy                           0.72     50428
   macro avg       0.58      0.71      0.57     50428
weighted avg       0.90      0.72      0.78     50428


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def     33406    12655
Real Default     1301     3066


In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, stratify=y, random_state=42)

# Probabilidades (idéntico conceptualmente)
y_test_proba = model.predict_proba(X_test)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_test, y_test_proba)

#optimal threshold based on expected loss
cost_FN = 10000  # default
cost_FP = 1000  # lost good client

losses = []

for t in thresholds:
    y_pred = (y_test_proba >= t).astype(int)

    FP = np.sum((y_test == 0) & (y_pred == 1))
    FN = np.sum((y_test == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_test_pred = (y_test_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_test, y_test_pred)

#Confusion matrix manual
TP = np.sum((y_test == 1) & (y_test_pred == 1))
FP = np.sum((y_test == 0) & (y_test_pred == 1))
TN = np.sum((y_test == 0) & (y_test_pred == 0))
FN = np.sum((y_test == 1) & (y_test_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f'used threshold: {adj_threshold}')
print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_test, y_test_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")


used threshold: 0.461111111111111
FPR: 0.2565
Precision (PPV): 0.2278
AUC validación: 0.84665
              precision    recall  f1-score   support

           0       0.97      0.74      0.84    138182
           1       0.23      0.80      0.35     13101

    accuracy                           0.75    151283
   macro avg       0.60      0.77      0.60    151283
weighted avg       0.91      0.75      0.80    151283


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def    102739    35443
Real Default     2647    10454


In [42]:
joblib.dump(model, '../models/catboost_new_dataset.pkl')

['../models/catboost_new_dataset.pkl']